In [1]:
from datasets import load_from_disk

kr_en = load_from_disk('../data/kr_en_aihub')
kr_cn = load_from_disk('../data/kr_cn_aihub')

In [2]:
pd_jp_train = kr_en['train'].to_pandas()
pd_jp_validation = kr_en['validation'].to_pandas()
pd_cn_train = kr_cn['train'].to_pandas()
pd_cn_validation = kr_cn['validation'].to_pandas()

In [3]:
pd_jp_train.insert(0, 'lang', 'en')
pd_cn_train.insert(0, 'lang', 'cn')
pd_jp_validation.insert(0, 'lang', 'en')
pd_cn_validation.insert(0, 'lang', 'cn')

In [4]:
pd_train = pd_jp_train.append(pd_cn_train)
pd_validation = pd_jp_validation.append(pd_cn_validation)

In [5]:
pd_train['text'] = pd_train['text'].apply(lambda x: x.replace(' ', ''))
pd_validation['text'] = pd_validation['text'].apply(lambda x: x.replace(' ', ''))

In [6]:
pd_train = pd_train[pd_train.duplicated(subset=['text'], keep=False)]
pd_validation = pd_validation[pd_validation.duplicated(subset=['text'], keep=False)]

In [7]:
from tqdm import tqdm

train_dict = {}

for idx, row in tqdm(pd_train.iterrows()):
  if row['text'] not in train_dict.keys():
    train_dict[row['text']] = {}
  train_dict[row['text']][row['lang']] = row['target']

250736it [00:14, 17235.51it/s]


In [8]:
from tqdm import tqdm

validation_dict = {}

for idx, row in tqdm(pd_validation.iterrows()):
  if row['text'] not in validation_dict.keys():
    validation_dict[row['text']] = {}
  validation_dict[row['text']][row['lang']] = row['target']

492it [00:00, 14916.42it/s]


In [9]:
del_train = []
del_validation = []
for key in train_dict.keys():
  if len(train_dict[key]) != 2:
    del_train.append(key)
for key in validation_dict.keys():
  if len(validation_dict[key]) != 2:
    del_validation.append(key)
for key in del_train:
  del train_dict[key]
for key in del_validation:
  del validation_dict[key]

In [10]:
len(train_dict.keys())

107066

In [11]:
train = {
  'id': [],
  'title': [],
  'text': [],
  'target': []
}

validation = {
  'id': [],
  'title': [],
  'text': [],
  'target': []
}

for idx, key in enumerate(train_dict.keys()):
  train['id'].append(str(idx))
  train['title'].append('aihub')
  train['text'].append(train_dict[key]['en'])
  train['target'].append(train_dict[key]['cn'])

for idx, key in enumerate(validation_dict.keys()):
  validation['id'].append(str(idx))
  validation['title'].append('aihub')
  validation['text'].append(validation_dict[key]['en'])
  validation['target'].append(validation_dict[key]['cn'])

In [12]:
from datasets import Dataset, DatasetDict

train_datasets = Dataset.from_dict(train)
validation_datasets = Dataset.from_dict(validation)

In [13]:
jp_cn_datasets = DatasetDict({
  'train': train_datasets,
  'validation': validation_datasets
})

In [15]:
jp_cn_datasets['train'][0]

{'id': '0',
 'title': 'aihub',
 'text': 'Nissan and Renault are at odds over the leadership of the combined system after former chairman Carlos Ghosn was arrested by Japanese prosecutors last November.',
 'target': '去年11月前会长卡洛斯·戈恩被日本检察机关逮捕后,日产和雷诺围绕联合体制的主导权产生了矛盾。'}

In [16]:
jp_cn_datasets.save_to_disk('../data/en_cn_aihub')